<a href="https://colab.research.google.com/github/moharamfatema/graduation-project/blob/main/src/spatiotemporal/muticlass/LAD2000_multi_244_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
import ipywidgets

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version

Num GPUs Available:  1
2.12.0


'3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import io
import matplotlib.pyplot as plt
import cv2
import imageio
#import medmnist
#import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models
# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
np.random.seed(SEED)

#Hyper parameters

In [ ]:
# DATA
DATASET_NAME = "LAD2000"
BATCH_SIZE = 64
PATH = '/content/gdrive/MyDrive/All extracted frames/Lad-Multi 224*224'
Extraction_Name = ''
# AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (224, 224, 15, 3)
NUM_CLASSES = 9

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 50

# TUBELET EMBEDDING
PATCH_SIZE = (25, 25, 15)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 16

#Loading data

In [ ]:
X_train, y_train = np.load(PATH + '/Train/X.npy', mmap_mode = 'r'), np.load(PATH + '/Train/y.npy', mmap_mode = 'r')
X_val, y_val = np.load(PATH + '/Val/X.npy', mmap_mode = 'r'), np.load(PATH + '/Val/y.npy', mmap_mode = 'r')
X_test, y_test = np.load(PATH + '/Test/X.npy', mmap_mode = 'r'), np.load(PATH + '/Test/y.npy', mmap_mode = 'r')

In [ ]:
from sklearn.utils import shuffle
#X_test, y_test = shuffle(X_test, y_test, random_state=42)
X_train, y_train = shuffle(X_train, y_train, random_state=42)
#X_val, y_val = shuffle(X_val,y_val,random_state=42)

ValueError: ignored

In [ ]:
y_train

array([11,  4,  4, ..., 13, 12,  6])

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

(4883, 15, 224, 224, 3) (4883, 1)
(3415, 15, 224, 224, 3) (3415, 1)
(4978, 15, 224, 224, 3) (4978, 1)


In [ ]:
X_train = np.moveaxis(X_train, 1, 3)
X_val = np.moveaxis(X_val, 1, 3)
X_test = np.moveaxis(X_test, 1, 3)

In [ ]:
y_train = to_categorical(y_train, 9)
y_val = to_categorical(y_val, 9)
y_test = to_categorical(y_test, 9)

In [ ]:
y_train[50]


6

#Data Generator

In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, x_in, y_in, batch_size=BATCH_SIZE, shuffle=True):
        # Initialization
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.x = x_in
        self.y = y_in
        self.datalen = len(y_in)
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # get batch indexes from shuffled indexes
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        x_batch = self.x[batch_indexes]
        y_batch = self.y[batch_indexes]
        return x_batch, y_batch

    def __len__(self):
        # Denotes the number of batches per epoch
        return self.datalen // self.batch_size

    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

training_generator = DataGenerator(X_train, y_train)
validation_generator = DataGenerator(X_val, y_val)

#ViVit Model

In [ ]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [ ]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

##Checkpoints and callbacks

In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/lad_224_multi/25_25_15_l16_8h.h5'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

In [ ]:
from keras.callbacks import ReduceLROnPlateau
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_lad_multi_of_vivit/checkpoint/'
checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

earlystopping = tf.keras.callbacks.EarlyStopping(
                     monitor="val_loss",
                     min_delta=0.025,
                     patience=5,
                     verbose=0,
                     mode="min",
                     baseline=None,
                     restore_best_weights=False
                 )

values = np.linspace(0.00001,LEARNING_RATE,14)[::-1].astype(np.float32)
boundaries = np.linspace(5, 45,13)[:values.shape[0]-1].astype(np.int32)

scheduler = keras.optimizers.schedules.PiecewiseConstantDecay(
    list(boundaries), list(values))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)

callbacks = [checkpoint, lr_scheduler, reduce_lr]

In [ ]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    input_shape = (224, 224, 15, 3)
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 75, 75, 15,  0           []                               
                                 3)]                                                              
                                                                                                  
 tubelet_embedding (TubeletEmbe  (None, 9, 128)      3600128     ['input_1[0][0]']                
 dding)                                                                                           
                                                                                                  
 positional_encoder (Positional  (None, 9, 128)      1152        ['tubelet_embedding[0][0]']      
 Encoder)                                                                                     

In [ ]:
#patch size = 25, 25, 15
# layers = 8
def run_experiment():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment()

Epoch 1/50
32/32 [==============================] - 63s 710ms/step - loss: 2.7504 - accuracy: 0.0977 - val_loss: 2.5859 - val_accuracy: 0.0977
Epoch 2/50
32/32 [==============================] - 10s 310ms/step - loss: 2.5967 - accuracy: 0.1104 - val_loss: 2.5648 - val_accuracy: 0.1523
Epoch 3/50
32/32 [==============================] - 9s 282ms/step - loss: 2.5748 - accuracy: 0.1196 - val_loss: 2.5811 - val_accuracy: 0.1133
Epoch 4/50
32/32 [==============================] - 8s 247ms/step - loss: 2.5511 - accuracy: 0.1509 - val_loss: 2.5818 - val_accuracy: 0.1211
Epoch 5/50
32/32 [==============================] - 9s 284ms/step - loss: 2.5436 - accuracy: 0.1489 - val_loss: 2.5718 - val_accuracy: 0.1250
Epoch 6/50
32/32 [==============================] - 9s 284ms/step - loss: 2.5374 - accuracy: 0.1401 - val_loss: 2.6083 - val_accuracy: 0.1406
Epoch 7/50
32/32 [==============================] - 8s 248ms/step - loss: 2.5126 - accuracy: 0.1646 - val_loss: 2.5334 - val_accuracy: 0.1055
Epoc

In [ ]:
# patch size = 25, 25, 15
# layers = 4
def run_experiment2():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment2()

Epoch 1/50
32/32 [==============================] - 66s 1s/step - loss: 2.7867 - accuracy: 0.0903 - val_loss: 2.6433 - val_accuracy: 0.1094
Epoch 2/50
32/32 [==============================] - 24s 762ms/step - loss: 2.5954 - accuracy: 0.1172 - val_loss: 2.6056 - val_accuracy: 0.1250
Epoch 3/50
32/32 [==============================] - 12s 361ms/step - loss: 2.5768 - accuracy: 0.1338 - val_loss: 2.6067 - val_accuracy: 0.0742
Epoch 4/50
32/32 [==============================] - 9s 277ms/step - loss: 2.5641 - accuracy: 0.1328 - val_loss: 2.6013 - val_accuracy: 0.1328
Epoch 5/50
32/32 [==============================] - 11s 334ms/step - loss: 2.5443 - accuracy: 0.1348 - val_loss: 2.5785 - val_accuracy: 0.0977
Epoch 6/50
32/32 [==============================] - 7s 230ms/step - loss: 2.5360 - accuracy: 0.1411 - val_loss: 2.6089 - val_accuracy: 0.0898
Epoch 7/50
32/32 [==============================] - 9s 269ms/step - loss: 2.5279 - accuracy: 0.1455 - val_loss: 2.5383 - val_accuracy: 0.1289
Epoch

KeyboardInterrupt: ignored

In [ ]:
# patch size = 15, 15, 15
# layers = 8
def run_experiment2():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment2()

Epoch 1/50
32/32 [==============================] - 26s 293ms/step - loss: 2.7039 - accuracy: 0.0967 - val_loss: 2.6248 - val_accuracy: 0.1406
Epoch 2/50
32/32 [==============================] - 7s 230ms/step - loss: 2.5795 - accuracy: 0.1343 - val_loss: 2.6194 - val_accuracy: 0.1016
Epoch 3/50
32/32 [==============================] - 9s 264ms/step - loss: 2.5642 - accuracy: 0.1318 - val_loss: 2.5983 - val_accuracy: 0.1016
Epoch 4/50
32/32 [==============================] - 9s 286ms/step - loss: 2.5524 - accuracy: 0.1333 - val_loss: 2.6105 - val_accuracy: 0.1211
Epoch 5/50
32/32 [==============================] - 8s 236ms/step - loss: 2.5549 - accuracy: 0.1455 - val_loss: 2.6095 - val_accuracy: 0.1562
Epoch 6/50
32/32 [==============================] - 9s 273ms/step - loss: 2.5319 - accuracy: 0.1533 - val_loss: 2.6042 - val_accuracy: 0.1133
Epoch 7/50
32/32 [==============================] - 8s 244ms/step - loss: 2.5220 - accuracy: 0.1606 - val_loss: 2.5917 - val_accuracy: 0.1328
Epoch

In [ ]:
# patch size = 15, 8, 8
# layers = 8
def run_experiment2():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment2()

Epoch 1/50
32/32 [==============================] - 40s 318ms/step - loss: 2.7293 - accuracy: 0.0933 - val_loss: 2.6842 - val_accuracy: 0.1133
Epoch 2/50
32/32 [==============================] - 8s 246ms/step - loss: 2.6043 - accuracy: 0.1201 - val_loss: 2.6391 - val_accuracy: 0.0859
Epoch 3/50
32/32 [==============================] - 9s 283ms/step - loss: 2.5757 - accuracy: 0.1318 - val_loss: 2.6395 - val_accuracy: 0.1289
Epoch 4/50
32/32 [==============================] - 9s 282ms/step - loss: 2.5770 - accuracy: 0.1279 - val_loss: 2.6283 - val_accuracy: 0.1094
Epoch 5/50
32/32 [==============================] - 8s 246ms/step - loss: 2.5675 - accuracy: 0.1196 - val_loss: 2.6227 - val_accuracy: 0.1094
Epoch 6/50
32/32 [==============================] - 9s 284ms/step - loss: 2.5316 - accuracy: 0.1362 - val_loss: 2.6033 - val_accuracy: 0.1133
Epoch 7/50
32/32 [==============================] - 9s 273ms/step - loss: 2.5189 - accuracy: 0.1543 - val_loss: 2.5923 - val_accuracy: 0.1367
Epoch

In [ ]:
# patch size = 20, 20, 15
# layers = 8
def run_experiment3():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment3()

Epoch 1/50
32/32 [==============================] - 40s 316ms/step - loss: 2.7222 - accuracy: 0.1133 - val_loss: 2.6319 - val_accuracy: 0.0898
Epoch 2/50
32/32 [==============================] - 9s 272ms/step - loss: 2.5806 - accuracy: 0.1230 - val_loss: 2.6108 - val_accuracy: 0.1445
Epoch 3/50
32/32 [==============================] - 8s 241ms/step - loss: 2.5658 - accuracy: 0.1240 - val_loss: 2.6002 - val_accuracy: 0.0820
Epoch 4/50
32/32 [==============================] - 9s 280ms/step - loss: 2.5595 - accuracy: 0.1299 - val_loss: 2.5897 - val_accuracy: 0.1250
Epoch 5/50
32/32 [==============================] - 9s 277ms/step - loss: 2.5463 - accuracy: 0.1445 - val_loss: 2.6098 - val_accuracy: 0.0938
Epoch 6/50
32/32 [==============================] - 12s 355ms/step - loss: 2.5350 - accuracy: 0.1484 - val_loss: 2.5709 - val_accuracy: 0.1133
Epoch 7/50
32/32 [==============================] - 9s 265ms/step - loss: 2.5219 - accuracy: 0.1611 - val_loss: 2.6118 - val_accuracy: 0.1094
Epoc

In [ ]:
# patch size = 8, 8, 15
# layers = 8
def run_experiment4():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment4()

Epoch 1/50
32/32 [==============================] - 40s 291ms/step - loss: 2.7453 - accuracy: 0.0942 - val_loss: 2.6132 - val_accuracy: 0.0820
Epoch 2/50
32/32 [==============================] - 11s 334ms/step - loss: 2.5861 - accuracy: 0.1318 - val_loss: 2.6434 - val_accuracy: 0.0977
Epoch 3/50
32/32 [==============================] - 9s 290ms/step - loss: 2.5817 - accuracy: 0.1240 - val_loss: 2.6282 - val_accuracy: 0.1211
Epoch 4/50
32/32 [==============================] - 8s 258ms/step - loss: 2.5516 - accuracy: 0.1421 - val_loss: 2.6031 - val_accuracy: 0.1719
Epoch 5/50
32/32 [==============================] - 9s 292ms/step - loss: 2.5473 - accuracy: 0.1440 - val_loss: 2.5935 - val_accuracy: 0.1445
Epoch 6/50
32/32 [==============================] - 10s 318ms/step - loss: 2.5251 - accuracy: 0.1406 - val_loss: 2.5792 - val_accuracy: 0.1484
Epoch 7/50
32/32 [==============================] - 8s 258ms/step - loss: 2.5070 - accuracy: 0.1582 - val_loss: 2.5574 - val_accuracy: 0.1094
Epo

In [ ]:
# patch size = 15, 15, 15
# layers = 16
def run_experiment3():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment3()

Epoch 1/50
32/32 [==============================] - 78s 406ms/step - loss: 2.6998 - accuracy: 0.0889 - val_loss: 2.6100 - val_accuracy: 0.0977
Epoch 2/50
32/32 [==============================] - 11s 353ms/step - loss: 2.5909 - accuracy: 0.1230 - val_loss: 2.6281 - val_accuracy: 0.0977
Epoch 3/50
32/32 [==============================] - 9s 289ms/step - loss: 2.5897 - accuracy: 0.1191 - val_loss: 2.5933 - val_accuracy: 0.1055
Epoch 4/50
32/32 [==============================] - 11s 339ms/step - loss: 2.5701 - accuracy: 0.1377 - val_loss: 2.6195 - val_accuracy: 0.1328
Epoch 5/50
32/32 [==============================] - 11s 344ms/step - loss: 2.5509 - accuracy: 0.1528 - val_loss: 2.5815 - val_accuracy: 0.1289
Epoch 6/50
32/32 [==============================] - 10s 299ms/step - loss: 2.5391 - accuracy: 0.1514 - val_loss: 2.6535 - val_accuracy: 0.1172
Epoch 7/50
32/32 [==============================] - 10s 306ms/step - loss: 2.5365 - accuracy: 0.1479 - val_loss: 2.6069 - val_accuracy: 0.1328


In [ ]:
# patch size = 15, 15, 15
# layers = 32
# heads = 8
def run_experiment5():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment5()

Epoch 1/50
32/32 [==============================] - 168s 1s/step - loss: 2.8427 - accuracy: 0.1030 - val_loss: 2.5964 - val_accuracy: 0.1328
Epoch 2/50
32/32 [==============================] - 38s 1s/step - loss: 2.5981 - accuracy: 0.1250 - val_loss: 2.6036 - val_accuracy: 0.1250
Epoch 3/50
32/32 [==============================] - 38s 1s/step - loss: 2.5801 - accuracy: 0.1221 - val_loss: 2.6051 - val_accuracy: 0.1172
Epoch 4/50
32/32 [==============================] - 38s 1s/step - loss: 2.5753 - accuracy: 0.1343 - val_loss: 2.6190 - val_accuracy: 0.1367
Epoch 5/50
32/32 [==============================] - 38s 1s/step - loss: 2.5680 - accuracy: 0.1440 - val_loss: 2.6086 - val_accuracy: 0.0938
Epoch 6/50
32/32 [==============================] - 38s 1s/step - loss: 2.5566 - accuracy: 0.1382 - val_loss: 2.6269 - val_accuracy: 0.0898
Epoch 7/50
32/32 [==============================] - 39s 1s/step - loss: 2.5368 - accuracy: 0.1426 - val_loss: 2.6084 - val_accuracy: 0.1484
Epoch 8/50
32/32 [=

In [ ]:
# patch size = 15, 15, 15
# layers = 16
# heads = 16
def run_experiment6():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment6()

Epoch 1/50
32/32 [==============================] - 104s 1s/step - loss: 2.7825 - accuracy: 0.0962 - val_loss: 2.6482 - val_accuracy: 0.1172
Epoch 2/50
32/32 [==============================] - 36s 1s/step - loss: 2.5906 - accuracy: 0.1421 - val_loss: 2.6298 - val_accuracy: 0.1133
Epoch 3/50
32/32 [==============================] - 37s 1s/step - loss: 2.5780 - accuracy: 0.1240 - val_loss: 2.6088 - val_accuracy: 0.0977
Epoch 4/50
32/32 [==============================] - 36s 1s/step - loss: 2.5712 - accuracy: 0.1406 - val_loss: 2.6321 - val_accuracy: 0.0938
Epoch 5/50
32/32 [==============================] - 36s 1s/step - loss: 2.5640 - accuracy: 0.1338 - val_loss: 2.6423 - val_accuracy: 0.1289
Epoch 6/50
32/32 [==============================] - 37s 1s/step - loss: 2.5619 - accuracy: 0.1289 - val_loss: 2.6100 - val_accuracy: 0.1289
Epoch 7/50
32/32 [==============================] - 37s 1s/step - loss: 2.5391 - accuracy: 0.1528 - val_loss: 2.6199 - val_accuracy: 0.0938
Epoch 8/50
32/32 [=

In [ ]:
def run_experiment10():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator)

    return model, history


model, history = run_experiment10()

Epoch 1/50
76/76 [==============================] - 552s 6s/step - loss: 3.0749 - accuracy: 0.1702 - val_loss: 2.4879 - val_accuracy: 0.3072
Epoch 2/50
76/76 [==============================] - 264s 3s/step - loss: 2.3608 - accuracy: 0.1902 - val_loss: 1.9737 - val_accuracy: 0.5473
Epoch 3/50
76/76 [==============================] - 261s 3s/step - loss: 2.1247 - accuracy: 0.1949 - val_loss: 2.0814 - val_accuracy: 0.3831
Epoch 4/50
76/76 [==============================] - 274s 4s/step - loss: 2.1070 - accuracy: 0.2116 - val_loss: 2.1304 - val_accuracy: 0.2133
Epoch 5/50
76/76 [==============================] - 259s 3s/step - loss: 2.0809 - accuracy: 0.2288 - val_loss: 2.1678 - val_accuracy: 0.1920
Epoch 6/50
76/76 [==============================] - 260s 3s/step - loss: 2.0451 - accuracy: 0.2521 - val_loss: 2.2172 - val_accuracy: 0.2025
Epoch 7/50
76/76 [==============================] - 272s 4s/step - loss: 2.0249 - accuracy: 0.2656 - val_loss: 2.2399 - val_accuracy: 0.2236
Epoch 8/50
76

In [ ]:
def run_experiment20():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
        model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])
        #model.load_weights(checkpoint_filepath)
        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator, callbacks=model_checkpoint)

    return model, history


model, history = run_experiment20()

Epoch 1/50
76/76 [==============================] - 514s 5s/step - loss: 3.5394 - accuracy: 0.1645 - val_loss: 2.3581 - val_accuracy: 0.5349
Epoch 2/50
76/76 [==============================] - 266s 4s/step - loss: 2.5009 - accuracy: 0.1822 - val_loss: 2.0406 - val_accuracy: 0.5292
Epoch 3/50
76/76 [==============================] - 269s 4s/step - loss: 2.1499 - accuracy: 0.1865 - val_loss: 1.9934 - val_accuracy: 0.5264
Epoch 4/50
76/76 [==============================] - 266s 4s/step - loss: 2.1299 - accuracy: 0.1916 - val_loss: 2.0143 - val_accuracy: 0.4819
Epoch 5/50
76/76 [==============================] - 257s 3s/step - loss: 2.1157 - accuracy: 0.1992 - val_loss: 2.1378 - val_accuracy: 0.3058
Epoch 6/50
76/76 [==============================] - 257s 3s/step - loss: 2.0960 - accuracy: 0.2136 - val_loss: 2.1005 - val_accuracy: 0.3941
Epoch 7/50
76/76 [==============================] - 258s 3s/step - loss: 2.0753 - accuracy: 0.2354 - val_loss: 2.0962 - val_accuracy: 0.3906
Epoch 8/50
76

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import f1_score

def eval_model(model,x,y):
#     print(model.evaluate(x.reshape(x.shape+(1,)),y_encoded))
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred,axis = -1)
    y_numbers = np.argmax(y,axis=-1)
   # f1_score(y_test, y_pred, average='micro')
    #print(f1_score)
    target_names = [
    'Crash',
    'Crowd',
    'Destroy',
    'Drop',
    'FallIntoWater',
    'Falling',
    'Fighting',
    'Fire',
    'Hurt',
    'Loitering',
    'Panic',
    'Thiefing',
    'Trampled',
    'Violence'
]
    labels = target_names
    tick_marks = np.arange(len(labels))

    print(classification_report(y_numbers, y_pred, target_names=target_names))

    z = tf.math.confusion_matrix(y_numbers,y_pred).numpy().astype(np.int64)
    z = np.around(z.astype('float') / z.sum(axis=1)[:, np.newaxis], decimals=3)

    x = target_names
    y = target_names

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                    #xaxis = dict(title='x'),
                    #yaxis = dict(title='x')
                    )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))

    # add colorbar
    fig['data'][0]['showscale'] = True
    fig.show()

In [ ]:
eval_model(model,X_test,y_test)

19/19 [==============================] - 2s 40ms/step
               precision    recall  f1-score   support

        Crash       0.00      0.00      0.00        32
        Crowd       0.00      0.00      0.00        60
      Destroy       0.00      0.00      0.00        44
         Drop       0.00      0.00      0.00        42
FallIntoWater       0.00      0.00      0.00        45
      Falling       0.00      0.00      0.00        34
     Fighting       0.00      0.00      0.00        60
         Fire       0.11      1.00      0.20        64
         Hurt       0.00      0.00      0.00        33
    Loitering       0.00      0.00      0.00        34
        Panic       0.00      0.00      0.00        32
     Thiefing       0.00      0.00      0.00        34
     Trampled       0.00      0.00      0.00        34
     Violence       0.00      0.00      0.00        40

     accuracy                           0.11       588
    macro avg       0.01      0.07      0.01       588
 weighted

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#3dcnn

###Note: for 3dcnn, do not run the "to categorical cell"

In [ ]:
dim = X_train[0].shape

In [ ]:
def create_model(dim):
  model = models.Sequential()
  # model.add(layers.BatchNormalization())
  model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform', data_format='channels_last', input_shape=dim))
  model.add(layers.MaxPooling3D((2, 2, 1)))
  # model.add(layers.BatchNormalization())
  #model.add(layers.Conv3D(32, kernel_size=(3, 3, 2), activation='relu', kernel_initializer='he_uniform'))
  #model.add(layers.MaxPooling3D((3, 3, 1)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform'))
  model.add(layers.MaxPooling3D((2, 2, 1)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Flatten())
  # model.add(layers.Dropout(0.5))
  #model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dense(1024, activation='relu'))
  # model.add(layers.BatchNormalization())
  #model.add(layers.Dropout(0.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  # model.add(layers.BatchNormalization())
  model.add(layers.Dense(14, activation='softmax'))
  return model

In [ ]:
model = create_model(dim)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 73, 73, 14, 32)    608       
                                                                 
 max_pooling3d (MaxPooling3D  (None, 36, 36, 14, 32)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 34, 34, 14, 32)    9248      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 17, 17, 14, 32)   0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 129472)            0         
                                                                 
 dense (Dense)               (None, 1024)              1

In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_lad_multi_of_cnn/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

In [ ]:
from keras.callbacks import ReduceLROnPlateau
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_lad_multi_of_cnn/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

earlystopping = tf.keras.callbacks.EarlyStopping(
                     monitor="val_loss",
                     min_delta=0.025,
                     patience=10,
                     verbose=0,
                     mode="min",
                     baseline=None,
                     restore_best_weights=False
                 )

values = np.linspace(0.00001,LEARNING_RATE,14)[::-1].astype(np.float32)
boundaries = np.linspace(5, 45,13)[:values.shape[0]-1].astype(np.int32)

scheduler = keras.optimizers.schedules.PiecewiseConstantDecay(
    list(boundaries), list(values))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)

callbacks = [model_checkpoint]

In [ ]:
dim

(75, 75, 14, 2)

In [ ]:
def run_experiment():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():

        model2 = create_model(dim)
        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model2.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )
       # model2.load_weights(checkpoint_filepath)
        # Train the model.
        history = model2.fit(X_train, y_train, batch_size = 64, epochs=50, validation_data=(X_val, y_val))

    return model2, history


model2, history = run_experiment()

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


33/33 [==============================] - 23s 254ms/step - loss: 2.6308 - accuracy: 0.0985 - top-5-accuracy: 0.4614 - val_loss: 2.6175 - val_accuracy: 0.1007 - val_top-5-accuracy: 0.4479
Epoch 2/50
33/33 [==============================] - 7s 209ms/step - loss: 2.6200 - accuracy: 0.1009 - top-5-accuracy: 0.4367 - val_loss: 2.6135 - val_accuracy: 0.1007 - val_top-5-accuracy: 0.4514
Epoch 3/50
33/33 [==============================] - 6s 194ms/step - loss: 2.6165 - accuracy: 0.1043 - top-5-accuracy: 0.4478 - val_loss: 2.6127 - val_accuracy: 0.1007 - val_top-5-accuracy: 0.4653
Epoch 4/50
33/33 [==============================] - 7s 210ms/step - loss: 2.6162 - accuracy: 0.1009 - top-5-accuracy: 0.4634 - val_loss: 2.6130 - val_accuracy: 0.1007 - val_top-5-accuracy: 0.4514
Epoch 5/50
33/33 [==============================] - 7s 198ms/step - loss: 2.6151 - accuracy: 0.1009 - top-5-accuracy: 0.4483 - val_loss: 2.6113 - val_accuracy: 0.1007 - val_top-5-accuracy: 0.4653
Epoch 6/50
33/33 [============

In [ ]:
model2.evaluate(X_test, y_test)

19/19 [==============================] - 1s 40ms/step - loss: 2.6082 - accuracy: 0.1088 - top-5-accuracy: 0.4643


[2.608229637145996, 0.10884353518486023, 0.4642857015132904]

In [ ]:
y_pred = model2.predict(X_test)

19/19 [==============================] - 1s 34ms/step


In [ ]:
np.asarray(y_pred).shape

(588, 14)

In [ ]:
y_pred[0]

array([0.43696383, 0.58891726, 0.5129307 , 0.4929061 , 0.51663417,
       0.45525852, 0.589747  , 0.600718  , 0.44906256, 0.4481016 ,
       0.43623903, 0.44894215, 0.44785562, 0.48917446], dtype=float32)

In [ ]:
y_pred = np.argmax(y_pred,axis = -1)

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import f1_score

def eval_model(model,x,y):
#     print(model.evaluate(x.reshape(x.shape+(1,)),y_encoded))
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred,axis = -1)
    #y_numbers = np.argmax(y,axis=-1)
   # f1_score(y_test, y_pred, average='micro')
    #print(f1_score)
    target_names = [
    'Crash',
    'Crowd',
    'Destroy',
    'Drop',
    'FallIntoWater',
    'Falling',
    'Fighting',
    'Fire',
    'Hurt',
    'Loitering',
    'Panic',
    'Thiefing',
    'Trampled',
    'Violence'
]
    labels = target_names
    tick_marks = np.arange(len(labels))

    print(classification_report(y_test, y_pred, target_names=target_names))

    z = tf.math.confusion_matrix(y_test,y_pred).numpy().astype(np.int64)
    z = np.around(z.astype('float') / z.sum(axis=1)[:, np.newaxis], decimals=3)

    x = target_names
    y = target_names

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                    #xaxis = dict(title='x'),
                    #yaxis = dict(title='x')
                    )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))

    # add colorbar
    fig['data'][0]['showscale'] = True
    fig.show()

In [ ]:
eval_model(model2,X_test,y_test)

19/19 [==============================] - 1s 32ms/step
               precision    recall  f1-score   support

        Crash       0.00      0.00      0.00        32
        Crowd       0.00      0.00      0.00        60
      Destroy       0.00      0.00      0.00        44
         Drop       0.00      0.00      0.00        42
FallIntoWater       0.00      0.00      0.00        45
      Falling       0.00      0.00      0.00        34
     Fighting       0.00      0.00      0.00        60
         Fire       0.11      1.00      0.20        64
         Hurt       0.00      0.00      0.00        33
    Loitering       0.00      0.00      0.00        34
        Panic       0.00      0.00      0.00        32
     Thiefing       0.00      0.00      0.00        34
     Trampled       0.00      0.00      0.00        34
     Violence       0.00      0.00      0.00        40

     accuracy                           0.11       588
    macro avg       0.01      0.07      0.01       588
 weighted

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.utils import shuffle
X2, y2 = shuffle(X_test, y_test, random_state=0)

In [ ]:
eval_model(model2,X2,y2)

19/19 [==============================] - 1s 33ms/step
               precision    recall  f1-score   support

        Crash       0.00      0.00      0.00        32
        Crowd       0.00      0.00      0.00        60
      Destroy       0.00      0.00      0.00        44
         Drop       0.00      0.00      0.00        42
FallIntoWater       0.00      0.00      0.00        45
      Falling       0.00      0.00      0.00        34
     Fighting       0.00      0.00      0.00        60
         Fire       0.11      1.00      0.20        64
         Hurt       0.00      0.00      0.00        33
    Loitering       0.00      0.00      0.00        34
        Panic       0.00      0.00      0.00        32
     Thiefing       0.00      0.00      0.00        34
     Trampled       0.00      0.00      0.00        34
     Violence       0.00      0.00      0.00        40

     accuracy                           0.11       588
    macro avg       0.01      0.07      0.01       588
 weighted

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#Decision Tree

#Evaluation

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import f1_score

def eval_model(model,x,y):
#     print(model.evaluate(x.reshape(x.shape+(1,)),y_encoded))
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred,axis = -1)
    #y_numbers = np.argmax(y,axis=-1)
   # f1_score(y_test, y_pred, average='micro')
    #print(f1_score)
    target_names = [
    'Crash',
    'Crowd',
    'Destroy',
    'Drop',
    'FallIntoWater',
    'Falling',
    'Fighting',
    'Fire',
    'Hurt',
    'Loitering',
    'Panic',
    'Thiefing',
    'Trampled',
    'Violence'
]
    labels = target_names
    tick_marks = np.arange(len(labels))

    print(classification_report(y_test, y_pred, target_names=target_names))

    z = tf.math.confusion_matrix(y_test,y_pred).numpy().astype(np.int64)
    z = np.around(z.astype('float') / z.sum(axis=1)[:, np.newaxis], decimals=3)

    x = target_names
    y = target_names

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                    #xaxis = dict(title='x'),
                    #yaxis = dict(title='x')
                    )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))

    # add colorbar
    fig['data'][0]['showscale'] = True
    fig.show()